In [1]:
%pip install mlx
%pip install numpy
%pip install opencv-python


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import mlx.core as mx
import time
import cv2
import os

In [3]:
### MLX

In [5]:
def darken_blend_8_mx(base, active):
  return mx.where(mx.greater(base, active), active, base)

In [6]:
def color_burn_8_mx(base, active):
  return mx.where(mx.equal(active, 0.0), 255.0, 255.0 - (255.0 - base) / active)

In [7]:
def lighten_blend_8_mx(base, active):
  return mx.where(mx.less(base, active), active, base)

In [8]:
def color_dodge_8_mx(base, active):
  return mx.where(mx.equal(active, 255.0), 255.0, base / (255.0 - active))

In [9]:
def overlay_blend_8_mx(base, active):
  return mx.where(mx.greater_equal(base, 128.0), 2 * base + base - 2 * base * base / 255.0 - 255.0, 2 * base * base / 255.0)

In [10]:
def multiply_blend_8_mx(base, active):
  return base * active / 255.0

In [11]:
def linear_burn_8_mx(base, active):
  return base + active - 255.0

In [12]:
def screen_blend_8_mx(base, active):
  return base + active - base * active / 255.0

In [13]:
def linear_dodge_8_mx(base, active):
  return base + active

In [14]:
### Single

In [15]:
def normal_blend_f_mx(base, active, opacity):
  return opacity * active + (1-opacity) * base

In [16]:
def normal_blend_8_mx(base, active, opacity):
  return opacity * active + (255.0 - opacity) * base

In [17]:
### Runner

In [18]:
rng = np.random.default_rng(1)

In [19]:
folder = "./data/"

img_files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

bases = []
actives = []

for _file in img_files:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    rnd = rng.random(img.shape, dtype = np.float32) * 255
    bases.append(img)
    actives.append(rnd)

In [20]:
def mat_runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = mx.array(bases[i], mx.float32)
        a = mx.array(actives[i], mx.float32)
        start_time = time.perf_counter()
        mx.eval(f(b, a))
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def vec_runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        base = bases[i].flatten()
        active = actives[i].flatten()
        b = mx.array(base, mx.float32)
        a = mx.array(active, mx.float32)
        opacity = mx.array(rng.random(1, dtype = np.float32), mx.float32)
        start_time = time.perf_counter()
        mx.eval(f(b, a, opacity))
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def timer(bases, actives, f, runner):
    runs = 5
    times = []
    for _ in range(runs):
        times.append(runner(bases, actives, f))
    times = np.array(times)
    print(f"{np.average(times)}ms +/- {np.std(times)}ms")

In [21]:
timer(bases, actives, darken_blend_8_mx, mat_runner)

6414.046255592257ms +/- 374.5294325176546ms


In [22]:
timer(bases, actives, color_burn_8_mx, mat_runner)

8064.282566960901ms +/- 295.1573672833318ms


In [23]:
timer(bases, actives, lighten_blend_8_mx, mat_runner)

7028.574208542705ms +/- 402.2918122230905ms


In [24]:
timer(bases, actives, color_dodge_8_mx, mat_runner)

8230.475757271051ms +/- 129.34228822621841ms


In [25]:
timer(bases, actives, overlay_blend_8_mx, mat_runner)

10926.442891173065ms +/- 220.33721278313888ms


In [26]:
timer(bases, actives, multiply_blend_8_mx, mat_runner)

7983.681849669665ms +/- 685.2647212963493ms


In [27]:
timer(bases, actives, linear_burn_8_mx, mat_runner)

7680.3849728778005ms +/- 653.2197472203173ms


In [28]:
timer(bases, actives, screen_blend_8_mx, mat_runner)

8242.720625549555ms +/- 212.17419665883756ms


In [29]:
timer(bases, actives, linear_dodge_8_mx, mat_runner)

7272.81727110967ms +/- 341.0646680882315ms


In [30]:
timer(bases, actives, normal_blend_f_mx, vec_runner)

167 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
timer(bases, actives, normal_blend_8_mx, vec_runner)

194 µs ± 24.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
